In [2]:
#  -------------------------------------------
"""
입출력 양식을 
준수해 주십시오

###  INPUT ###
import pandas as pd
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv')

{
    
        Write codes...
    
    Training model name : model
    ex) y_pred = model.predict(X_test)
    

}


### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)
"""

'\n입출력 양식을 \n준수해 주십시오\n\n###  INPUT ###\nimport pandas as pd\ninput_data = pd.read_csv(\'2023_smartFarm_AI_hackathon_dataset.csv\')\n\n{\n    \n        Write codes...\n    \n    Training model name : model\n    ex) y_pred = model.predict(X_test)\n    \n\n}\n\n\n### OUTPUT ###\nprint("RMSE:", rmse)\nprint("R2_score:", r2score)\n'

In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.models import load_model
from keras.callbacks import EarlyStopping

from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm

import numpy.random as nr
import scipy.stats as ss
import math

import time
from keras.callbacks import EarlyStopping  
%matplotlib inline

In [98]:
from tensorflow.keras.models import load_model
from sklearn import linear_model

# def print_metrics(y_true, y_predicted, n_parameters):
#     ## First compute R^2 and the adjusted R^2
#     r2 = sklm.r2_score(y_true, y_predicted)
#     r2_adj = r2 - (n_parameters - 1)/(y_true.shape[0] - n_parameters) * (1 - r2)


#     ## Print the usual metrics and the R^2 values
#     # print('Mean Square Error      = ' + str(sklm.mean_squared_error(y_true, y_predicted)))
#     print('Root Mean Square Error = ' + str(math.sqrt(sklm.mean_squared_error(y_true, y_predicted))))
#     # print('Mean Absolute Error    = ' + str(sklm.mean_absolute_error(y_true, y_predicted)))
#     # print('Median Absolute Error  = ' + str(sklm.median_absolute_error(y_true, y_predicted)))
#     print('R^2                    = ' + str(r2*100))
#     # print('Adjusted R^2           = ' + str(r2_adj))

#     return sklm.mean_squared_error(y_true, y_predicted), r2

In [99]:
def LSTM_input2(df):
    Features = df
    x_train = []
    y_train = []
    x_test = []
    y_test = []

    n_future = 1  # Number of days we want to predict into the future
    n_past = 16  # Number of past days we want to use to predict the future

    labels = np.array([row[0] for row in Features])

    print(Features.shape)
    indx = range(Features.shape[0])
    indx = ms.train_test_split(indx, test_size=0.2, shuffle=False)
    # print(indx)
    # Features_train_x = sc.fit_transform(Features[0:len(indx[0]) + 1, 1:16])
    # Features_train_y = sc.fit_transform(Features[0:len(indx[0]) + 1, 0:2])

    # print(Features[0:len(indx[0]) + 1, 1:16])
    Features_train_x = sc.fit_transform(Features[0:len(indx[0]) + 1, 1:16])
    Features_train_y = sc.fit_transform(Features[0:len(indx[0]) + 1, 0:1])  # 2개의 y 변수를 포함하도록 수정\
    # print(Features_train_x.shape)
    # print(Features_train_y.shape)
    Features_train = np.hstack([Features_train_y, Features_train_x])

    Features_test_x = sc.fit_transform(Features[len(indx[0]) - n_past:len(Features), 1:16])
    Features_test_y = sc.fit_transform(Features[len(indx[0]) - n_past:len(Features), 0:1])  # 2개의 y 변수를 포함하도록 수정

    Features_test = np.hstack([Features_test_y, Features_test_x])

    for i in range(n_past, len(indx[0]) - n_future + 1):
        x_train.append(Features_train[i - n_past:i, 0:Features.shape[1]])  # 모든 feature를 포함하도록 수정
        y_train.append(Features_train[i:i + n_future, 0])  # 2개의 y 변수를 포함하도록 수정

    for i in range(n_past, n_past + len(indx[1])):
        x_test.append(Features_test[i - n_past:i, 0:Features.shape[1]])  # 모든 feature를 포함하도록 수정
        y_test.append(Features_test[i:i + n_future, 0:1])  # 2개의 y 변수를 포함하도록 수정

    x_train, y_train, x_test, y_test = np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)
    
    return x_train, y_train, x_test, y_test

In [93]:
# def LSTM_input(df):
    
#     Features=df
#     x_train = []
#     y_train = []
#     x_test = []
#     y_test = []

#     n_future = 1  # Number of days we want top predict into the future
#     n_past = 16 # Number of past days we want to use to predict the future

#     labels = np.array([row[0] for row in Features])
#     # print(labels.shape)
    
#     indx = range(Features.shape[0])
#     indx = ms.train_test_split(indx, test_size = 0.2, shuffle=False)

#     #Features_train = sc.fit_transform(Features[0:len(indx[0])+1,:])
#     #Features_test = sc.fit_transform(Features[len(indx[0])-n_past:len(Features),:])
#     Features_train_x = sc.fit_transform(Features[0:len(indx[0])+1,1:16])
#     Features_train_y = sc.fit_transform(Features[0:len(indx[0])+1,0:1])
#     print(Features_train_y)
#     Features_train = np.hstack([Features_train_y,Features_train_x])

#     Features_test_x = sc.fit_transform(Features[len(indx[0])-n_past:len(Features),1:16])
#     Features_test_y = sc.fit_transform(Features[len(indx[0])-n_past:len(Features),0:1])

#     Features_test = np.hstack([Features_test_y,Features_test_x])

#     for i in range(n_past, len(indx[0]) - n_future +1):
#         x_train.append(Features_train[i - n_past:i, 0:Features.shape[1]])
#         y_train.append(Features_train[i:i + n_future ,0])

#     for i in range(n_past, n_past+len(indx[1])):
#         x_test.append(Features_test[i - n_past :i, 0:Features.shape[1]])
#         y_test.append(Features_test[i:i + n_future,0])

#     x_train, y_train = np.array(x_train), np.array(y_train)
#     x_test, y_test = np.array(x_test), np.array(y_test)
#     # comment out this box and uncomment load_model to load saved model
#     ###################################################################################

#     return x_train, y_train, x_test, y_test

In [100]:
def prepro(df):
    input_data = df
    # input_data=input_data.groupby("frmDist")
    # flist = input_data["frmDist"].unique()

    # for p in flist[:1]:
    #     input_data.get_group(p)
    input_data.loc[input_data["outtrn_cumsum"]==0,"outtrn_cumsum"]=None
    input_data["outtrn_cumsum"]=input_data["outtrn_cumsum"].fillna(method='ffill' or 'pad')
    # input_data=input_data.dropna(subset=['outtrn_cumsum'])
    input_data.loc[input_data["HeatingEnergyUsage_cumsum"]==0,"HeatingEnergyUsage_cumsum"]=None
    input_data["HeatingEnergyUsage_cumsum"]=input_data["HeatingEnergyUsage_cumsum"].fillna(method='ffill' or 'pad')
    
    # 0 값을 NaN으로 바꾸기
    input_data = input_data.replace(0, np.nan)
    # frtstCo(열매 수) 누적값으로 바꾸기
    input_data['frtstCo'] = input_data['frtstCo'].cumsum()
    input_data['outTp_minus_inTp'] = input_data['outTp_minus_inTp'].cumsum()
    # input_data=input_data.dropna(subset=['HeatingEnergyUsage_cumsum'])
    input_data = input_data.interpolate()
    input_data = input_data.replace(np.nan,0)
    
    return input_data

In [101]:
training_set

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.31527426e+05, 0.00000000e+00, 2.30969907e+01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.51254322e+05, 0.00000000e+00, 2.30969907e+01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.51254322e+05, 0.00000000e+00, 2.30969907e+01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [102]:
import warnings
warnings.filterwarnings(action='ignore')

# ------------------------------------------------


#      SAMPLE MODEL


# ------------------------------------------------


###  INPUT ###
"""

Read CSV files from the given list of file paths and return DataFrames.

"""

import pandas as pd

input_data = pd.read_csv("2023_smartFarm_AI_hackathon_dataset.csv")
#  -------------------------------------------
###    Write codes...  ###
#      EXAMPLE CODE      #
df_delna = input_data.drop(['daysuplyqy', 'lefstalklt', 'frtstSetCo', 'pllnLt', 'flanJnt', 'hvstJnt', 'flwrCo', 'frtstJnt'], axis=1)
df_delna.sort_values(by=["frmDist","date"],ascending=True,inplace=True)
jflist=df_delna["frmDist"].unique()
fj_env=df_delna.groupby("frmDist")

# Read CSV files from the lists of file paths


# Now you have lists of DataFrames for each type of data 

import numpy as np
from sklearn.model_selection import train_test_split

# ... (Data preprocessing code here)

flist = input_data["frmDist"].unique()
input_data=input_data.drop(['daysuplyqy', 'lefstalklt', 'frtstSetCo', 'pllnLt', 'flanJnt', 'hvstJnt', 'flwrCo', 'frtstJnt'],axis=1)
input_data.sort_values(by=["frmDist","date"],ascending=True,inplace=True)

input_data = input_data.copy()

input_data['inTp'].replace(0, np.nan, inplace=True)
input_data['outTp'].replace(0, np.nan, inplace=True)

input_data['mmdd'] = input_data['date'].astype(str).str[4:]

# 'mmdd'를 기준으로 그룹화하여 변수값의 평균을 계산하고 대체
input_data['inTp'].fillna(input_data.groupby('mmdd')['inTp'].transform('mean'), inplace=True)
input_data['outTp'].fillna(input_data.groupby('mmdd')['outTp'].transform('mean'), inplace=True)

# input_data_r.drop(['mmdd'], axis=1, inplace=True)
input_data['inTp'].fillna(0, inplace=True)
input_data['outTp'].fillna(0, inplace=True)

# 뺀 변수 만들기
input_data['outTp_minus_inTp'] = input_data['inTp'] - input_data['outTp']

empty_df = pd.DataFrame()
for p in flist:
    df=fj_env.get_group(p)
    df=df.reindex(columns=["date","outtrn_cumsum",'HeatingEnergyUsage_cumsum',"inTp","outTp","inHd","inCo2","outWs","acSlrdQy","ec","ph",
                        "frtstGrupp","flanGrupp","frtstCo", 'tcdmt', 'frmhsFclu', 'hvstGrupp',
                        'grwtLt', 'fcluHg', 'lefLt', 'hvstCo', 'lefCunt',
                        'lefBt', 'stemThck', 'frmAr', 'frmDov','outTp_minus_inTp'
                        ])
    df=prepro(df)
    df=df.set_index("date")
    empty_df=pd.concat([empty_df, df], axis = 0)
    
# Initialize and train the LinearRegression model
empty_df=empty_df.reset_index()
dataset_train_actual = empty_df.copy()
dataset_train_actual['date']= dataset_train_actual['date'].astype('str')
dataset_train_actual['date']=pd.to_datetime(dataset_train_actual['date'])
dataset_train_timeindex = dataset_train_actual.set_index('date')

#---------------------------------------------------------------------
### 변수 나누는 부분
dataset_train = dataset_train_actual.copy()

# # training and predictions 포함된 열 선택
cols = list(dataset_train)[1:25]
# print(cols)
# # # datelist_train 그림그리기위한 시간 열
datelist_train = list(dataset_train['date'])
datelist_train = [date for date in datelist_train]
dataset_train = dataset_train[cols].astype(str)

# # datelist_train float 타입으로 바꿔주기
dataset_train = dataset_train.astype(float)

# # 여러 feature값 (predictors)
training_set = dataset_train.values
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()

# ------------------------------------------------
# y 종속변수 2개 지정 
print(dataset_train_actual.shape)
train_set_scale = training_set[:,2:]

train_set_scale=np.append(train_set_scale[:, 0:2],train_set_scale,axis=1)
train_set_scale

# x_train, y_train, x_test, y_test = LSTM_input(train_set_scale)
x_train, y_train, x_test, y_test = LSTM_input2(train_set_scale)
    
# design network
model = Sequential()
model.add(LSTM(int(32), input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(.3))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train,y_train,
          epochs=1,
          batch_size=32,
          validation_data=(x_test, y_test),
          verbose=1,
          shuffle=False,
          # callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)]
         )
y_pred = sc.inverse_transform(model.predict(x_test))

# Calculate RMSE between the predictions and actual 'y' values

def calculate_rmse(targets, predictions):
    """
    Calculate the Root Mean Squared Error (RMSE) between predicted and target values.

    :param predictions: Predicted values.
    :type predictions: array-like
    :param targets: Target values.
    :type targets: array-like
    :return: RMSE value.
    :rtype: float
    """
    from sklearn.metrics import mean_squared_error
    return np.sqrt(mean_squared_error(targets, predictions))


# Calculate r2_score between the predictions and actual 'y' values

def calculate_R2_score(y_test,y_score):
    # n_parameters=x_train.shape[1]
    # y_score = sc.inverse_transform(model.predict(x_test))
    # print(y_score.shape)
    # rmse, r2score = print_metrics(sc.inverse_transform(y_test), y_score, n_parameters)
    # print("RMSE:", rmse)
    # print("R2_score:", r2score)
    from sklearn.metrics import r2_score
    r2 = sklm.r2_score(y_test, y_score)
    return r2

# 평가지표 계산을 위한 shape 변형
y_train=y_train.reshape(-1,1)
y_test=y_test.reshape(-1,1)

y_test=pd.DataFrame(sc.inverse_transform(y_test))
rmse = calculate_rmse(y_test, y_pred)
r2score = calculate_R2_score(y_test, y_pred)

# ------------------------------------------------
### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)


(84840, 27)
(84840, 24)
531/531 [==============================] - 4s 6ms/step
RMSE: 3.1789816289174673
R2_score: 0.7832291970717024


In [90]:
y_test

,0
0,22.492646
1,25.323035
2,21.576610
3,21.266500
4,20.124594
...,...
16963,23.096991
16964,23.096991
16965,23.096991
16966,23.096991


In [91]:
y_pred

array([[20.901802],
       [21.079952],
       [21.90747 ],
       ...,
       [22.041739],
       [21.99608 ],
       [21.97025 ]], dtype=float32)

In [32]:

with pd.ExcelWriter('y.xlsx') as writer:
    pd.DataFrame(sc.inverse_transform(y_pred)).to_excel(writer, sheet_name='y_pred')
    pd.DataFrame(sc.inverse_transform(y_test)).to_excel(writer, sheet_name='y_test')

Exception ignored in: <function ZipFile.__del__ at 0x7fec07e0f1f0>
Traceback (most recent call last):
  File "/home/jupyter-kty7613/.conda/envs/kty/lib/python3.9/zipfile.py", line 1816, in __del__
    self.close()
  File "/home/jupyter-kty7613/.conda/envs/kty/lib/python3.9/zipfile.py", line 1833, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file
